# NARDL-Fourier: Complete Tutorial
## Nonlinear ARDL with Fourier Approximation & Bootstrap Cointegration

**Author:** Dr. Merwan Roudane  
**Email:** merwanroudane920@gmail.com  
**GitHub:** https://github.com/merwanroudane/fnardl  
**Version:** 1.0.1

---

This comprehensive notebook demonstrates the full workflow for analyzing asymmetric cointegration relationships using the NARDL-Fourier library with real economic data.

## Table of Contents

1. [Environment Setup](#1-environment-setup)
2. [Data Loading & Exploration](#2-data-loading--exploration)
3. [Data Cleaning & Preprocessing](#3-data-cleaning--preprocessing)
4. [Exploratory Data Analysis](#4-exploratory-data-analysis)
5. [Stationarity Testing](#5-stationarity-testing)
6. [Standard NARDL Model](#6-standard-nardl-model)
7. [Fourier NARDL Model](#7-fourier-nardl-model)
8. [Bootstrap NARDL Model](#8-bootstrap-nardl-model)
9. [Model Comparison](#9-model-comparison)
10. [Diagnostic Tests](#10-diagnostic-tests)
11. [Visualizations](#11-visualizations)
12. [Results Interpretation](#12-results-interpretation)
13. [Export Results](#13-export-results)

---
## 1. Environment Setup

In [ ]:
# Install required packages (uncomment if needed)
# !pip install nardl-fourier watermark

# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# NARDL-Fourier library
from nardl_fourier import NARDL, FourierNARDL, BootstrapNARDL
from nardl_fourier import ResultsTable, NARDLPlots
from nardl_fourier.diagnostics import run_all_diagnostics

# Statsmodels for unit root tests
from statsmodels.tsa.stattools import adfuller, kpss

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Seaborn settings
sns.set_palette('husl')

print("Environment ready!")

In [ ]:
# Watermark for reproducibility
%load_ext watermark
%watermark -a "Dr. Merwan Roudane" -u -d -v -p numpy,pandas,scipy,statsmodels,matplotlib,nardl_fourier

---
## 2. Data Loading & Exploration

We use the **Kilian (2009)** dataset containing monthly data on:
- `dprod`: Percent change in global crude oil production
- `rea`: Index of real economic activity
- `rpo`: Real price of oil

In [ ]:
# Load the Kilian dataset
data = pd.read_excel('Killian_data.xlsx')

# Display basic info
print("="*60)
print("KILIAN (2009) DATASET")
print("="*60)
print(f"\nShape: {data.shape[0]} observations x {data.shape[1]} variables")
print(f"\nVariables: {data.columns.tolist()}")
print(f"\nDate Range: {data['date'].min():.2f} to {data['date'].max():.2f}")
print("\n" + "="*60)

In [ ]:
# First 10 rows
print("First 10 observations:")
data.head(10)

In [ ]:
# Last 10 rows
print("Last 10 observations:")
data.tail(10)

In [ ]:
# Data types
print("Data Types:")
print(data.dtypes)

---
## 3. Data Cleaning & Preprocessing

In [ ]:
# Check for missing values
print("Missing Values:")
print(data.isnull().sum())
print(f"\nTotal missing: {data.isnull().sum().sum()}")

In [ ]:
# Check for duplicates
print(f"Duplicate rows: {data.duplicated().sum()}")

In [ ]:
# Create proper datetime index
def convert_date(x):
    year = int(x)
    month = int(round((x - year) * 100))
    return pd.Timestamp(year=year, month=month, day=1)

data['datetime'] = data['date'].apply(convert_date)
data = data.set_index('datetime')

print("Data with datetime index:")
data.head()

In [ ]:
# Reset index for NARDL models (they expect DataFrame with numeric index)
df = data.reset_index(drop=True)
print(f"Working DataFrame shape: {df.shape}")
df.head()

---
## 4. Exploratory Data Analysis

In [ ]:
# Descriptive statistics
print("Descriptive Statistics:")
print("="*80)
desc = df[['dprod', 'rea', 'rpo']].describe().T
desc['skewness'] = df[['dprod', 'rea', 'rpo']].skew()
desc['kurtosis'] = df[['dprod', 'rea', 'rpo']].kurtosis()
desc.round(4)

In [ ]:
# Time series plots
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

variables = ['dprod', 'rea', 'rpo']
titles = ['Global Oil Production Growth (%)', 'Real Economic Activity Index', 'Real Price of Oil']
colors = ['#2ecc71', '#3498db', '#e74c3c']

for i, (var, title, color) in enumerate(zip(variables, titles, colors)):
    axes[i].plot(data.index, data[var], color=color, linewidth=1.2)
    axes[i].set_title(title, fontweight='bold')
    axes[i].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    axes[i].fill_between(data.index, data[var], 0, alpha=0.3, color=color)
    axes[i].set_ylabel(var)

axes[2].set_xlabel('Date')
plt.suptitle('Kilian (2009) Oil Market Variables', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('time_series_plots.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Distribution plots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, (var, color) in enumerate(zip(variables, colors)):
    sns.histplot(df[var], kde=True, ax=axes[i], color=color, alpha=0.7)
    axes[i].set_title(f'Distribution of {var}', fontweight='bold')
    axes[i].axvline(df[var].mean(), color='red', linestyle='--', label=f'Mean: {df[var].mean():.2f}')
    axes[i].legend()

plt.suptitle('Variable Distributions', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation matrix
fig, ax = plt.subplots(figsize=(8, 6))
corr = df[['dprod', 'rea', 'rpo']].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, annot=True, fmt='.3f', cmap='RdBu_r', center=0, 
            square=True, linewidths=2, ax=ax, vmin=-1, vmax=1)
ax.set_title('Correlation Matrix', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nCorrelation Matrix:")
print(corr.round(4))

In [ ]:
# Scatter plot matrix
fig = sns.pairplot(df[['dprod', 'rea', 'rpo']], diag_kind='kde', 
                   plot_kws={'alpha': 0.5, 's': 20}, corner=True)
fig.fig.suptitle('Pairwise Relationships', y=1.02, fontweight='bold')
plt.savefig('pairplot.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 5. Stationarity Testing

Before applying NARDL, we verify that variables are I(0) or I(1), but not I(2).

In [ ]:
def stationarity_tests(series, name):
    """Run ADF and KPSS tests on a series."""
    results = {'Variable': name}
    
    # ADF Test (H0: unit root)
    adf_result = adfuller(series.dropna(), autolag='AIC')
    results['ADF Stat'] = adf_result[0]
    results['ADF p-value'] = adf_result[1]
    results['ADF Decision'] = 'Stationary' if adf_result[1] < 0.05 else 'Non-stationary'
    
    # KPSS Test (H0: stationary)
    kpss_result = kpss(series.dropna(), regression='c', nlags='auto')
    results['KPSS Stat'] = kpss_result[0]
    results['KPSS p-value'] = kpss_result[1]
    results['KPSS Decision'] = 'Non-stationary' if kpss_result[1] < 0.05 else 'Stationary'
    
    return results

# Test all variables in levels
print("="*80)
print("STATIONARITY TESTS - LEVELS")
print("="*80)

level_results = []
for var in ['dprod', 'rea', 'rpo']:
    level_results.append(stationarity_tests(df[var], var))

level_df = pd.DataFrame(level_results)
print(level_df.to_string(index=False))

In [ ]:
# Test first differences
print("\n" + "="*80)
print("STATIONARITY TESTS - FIRST DIFFERENCES")
print("="*80)

diff_results = []
for var in ['dprod', 'rea', 'rpo']:
    diff_results.append(stationarity_tests(df[var].diff(), f'd_{var}'))

diff_df = pd.DataFrame(diff_results)
print(diff_df.to_string(index=False))

In [ ]:
# Summary interpretation
print("\n" + "="*80)
print("INTEGRATION ORDER SUMMARY")
print("="*80)
print("""
Variable     Order    Interpretation
--------     -----    --------------
dprod        I(0)     Stationary in levels (production growth)
rea          I(1)     Non-stationary, stationary after differencing
rpo          I(1)     Non-stationary, stationary after differencing

✓ All variables are I(0) or I(1) → NARDL applicable!
""")

---
## 6. Standard NARDL Model

We estimate the relationship between real oil prices (`rpo`) and its determinants, with asymmetric effects of real economic activity (`rea`).

In [ ]:
# Fit Standard NARDL model
print("="*80)
print("STANDARD NARDL MODEL ESTIMATION")
print("="*80)

nardl_model = NARDL(
    data=df,
    depvar='rpo',              # Real price of oil
    exog_vars=['dprod'],       # Oil production (symmetric)
    decomp_vars=['rea'],       # Real economic activity (asymmetric)
    maxlag=8,                  # Maximum lag
    ic='AIC'                   # Information criterion
)

print(nardl_model.summary())

In [ ]:
# Long-run coefficients
print("\n" + "="*80)
print("LONG-RUN COEFFICIENTS")
print("="*80)
print(nardl_model.long_run_table())

In [ ]:
# Short-run coefficients
print("\n" + "="*80)
print("SHORT-RUN COEFFICIENTS")
print("="*80)
print(nardl_model.short_run_table())

In [ ]:
# Asymmetry tests
print("\n" + "="*80)
print("ASYMMETRY TESTS")
print("="*80)

lr_test = nardl_model.wald_lr_asymmetry()
sr_test = nardl_model.wald_sr_asymmetry()

print(f"\nLong-Run Asymmetry (H0: L+ = L-):")
print(f"   Wald statistic: {lr_test['statistic']:.4f}")
print(f"   p-value: {lr_test['pvalue']:.4f}")
print(f"   Decision: {'Reject H0 (Asymmetric)' if lr_test['pvalue'] < 0.05 else 'Fail to Reject H0 (Symmetric)'}")

print(f"\nShort-Run Asymmetry (H0: Σπ+ = Σπ-):")
print(f"   Wald statistic: {sr_test['statistic']:.4f}")
print(f"   p-value: {sr_test['pvalue']:.4f}")
print(f"   Decision: {'Reject H0 (Asymmetric)' if sr_test['pvalue'] < 0.05 else 'Fail to Reject H0 (Symmetric)'}")

In [ ]:
# Bounds test
print("\n" + "="*80)
print("PSS BOUNDS TEST FOR COINTEGRATION")
print("="*80)

bounds = nardl_model.bounds_test()
print(f"\nF-statistic: {bounds['f_statistic']:.4f}")
print(f"\nCritical Values:")
print(f"   10%: I(0) = {bounds['cv_I0_10pct']:.3f}, I(1) = {bounds['cv_I1_10pct']:.3f}")
print(f"    5%: I(0) = {bounds['cv_I0_5pct']:.3f}, I(1) = {bounds['cv_I1_5pct']:.3f}")
print(f"    1%: I(0) = {bounds['cv_I0_1pct']:.3f}, I(1) = {bounds['cv_I1_1pct']:.3f}")
print(f"\nDecision: {bounds['decision']}")

---
## 7. Fourier NARDL Model

Fourier NARDL captures smooth structural breaks without specifying break dates.

In [ ]:
# Fit Fourier NARDL model
print("="*80)
print("FOURIER NARDL MODEL ESTIMATION")
print("="*80)

fnardl_model = FourierNARDL(
    data=df,
    depvar='rpo',
    exog_vars=['dprod'],
    decomp_vars=['rea'],
    maxlag=8,
    max_freq=3,               # Test frequencies k=1,2,3
    ic='AIC'
)

print(f"\nOptimal Fourier Frequency: k* = {fnardl_model.best_freq}")
print(fnardl_model.summary())

In [ ]:
# Frequency comparison
print("\n" + "="*80)
print("FOURIER FREQUENCY COMPARISON")
print("="*80)

for k, ic_val in fnardl_model.frequency_comparison.items():
    marker = " ← OPTIMAL" if k == fnardl_model.best_freq else ""
    print(f"   k = {k}: AIC = {ic_val:.4f}{marker}")

In [ ]:
# Long-run coefficients from Fourier NARDL
print("\n" + "="*80)
print("FOURIER NARDL - LONG-RUN COEFFICIENTS")
print("="*80)
print(fnardl_model.long_run_table())

---
## 8. Bootstrap NARDL Model

Bootstrap NARDL eliminates the inconclusive zone problem of PSS bounds tests.

In [ ]:
# Fit Bootstrap NARDL model
print("="*80)
print("BOOTSTRAP NARDL MODEL ESTIMATION")
print("="*80)
print("Running bootstrap (this may take a moment)...\n")

bnardl_model = BootstrapNARDL(
    data=df,
    depvar='rpo',
    exog_vars=['dprod'],
    decomp_vars=['rea'],
    maxlag=8,
    n_bootstrap=1000,         # Bootstrap replications
    seed=42                   # For reproducibility
)

print(bnardl_model.summary())

In [ ]:
# Bootstrap cointegration tests
print("\n" + "="*80)
print("BOOTSTRAP COINTEGRATION TESTS (Bertelli et al., 2022)")
print("="*80)

tests = bnardl_model.cointegration_tests()

for test_name, results in tests.items():
    decision = '✓ Reject H₀' if results['reject'] else '✗ Fail to Reject H₀'
    print(f"\n{test_name}:")
    print(f"   Statistic: {results['statistic']:.4f}")
    print(f"   Bootstrap CV (5%): {results['critical_value']:.4f}")
    print(f"   Bootstrap p-value: {results['pvalue']:.4f}")
    print(f"   Decision: {decision}")

print("\n" + "="*80)
print(f"CONCLUSION: {bnardl_model.cointegration_decision()}")
print("="*80)

---
## 9. Model Comparison

In [ ]:
# Compare all three models
print("="*80)
print("MODEL COMPARISON")
print("="*80)

comparison = pd.DataFrame({
    'Model': ['Standard NARDL', 'Fourier NARDL', 'Bootstrap NARDL'],
    'AIC': [nardl_model.aic, fnardl_model.aic, bnardl_model.aic],
    'BIC': [nardl_model.bic, fnardl_model.bic, bnardl_model.bic],
    'Adj. R²': [nardl_model.adj_rsquared, fnardl_model.adj_rsquared, bnardl_model.adj_rsquared],
    'Observations': [nardl_model.nobs, fnardl_model.nobs, bnardl_model.nobs]
})

# Add best model indicator
comparison['Best AIC'] = comparison['AIC'] == comparison['AIC'].min()
comparison['Best Adj. R²'] = comparison['Adj. R²'] == comparison['Adj. R²'].max()

print(comparison.to_string(index=False))

---
## 10. Diagnostic Tests

Comprehensive diagnostic tests for the best model.

In [ ]:
# Run all diagnostics on the Fourier NARDL model
print("="*80)
print("DIAGNOSTIC TESTS (Fourier NARDL)")
print("="*80)

diagnostics = run_all_diagnostics(fnardl_model)

# Normality tests
print("\n--- NORMALITY TESTS ---")
jb = diagnostics['normality']['jarque_bera']
print(f"Jarque-Bera: Stat={jb['statistic']:.4f}, p-value={jb['pvalue']:.4f}")

# Serial correlation tests
print("\n--- SERIAL CORRELATION TESTS ---")
bg = diagnostics['serial_correlation']['breusch_godfrey']
dw = diagnostics['serial_correlation']['durbin_watson']
print(f"Breusch-Godfrey: Stat={bg['statistic']:.4f}, p-value={bg['pvalue']:.4f}")
print(f"Durbin-Watson: {dw['statistic']:.4f}")

# Heteroskedasticity tests
print("\n--- HETEROSKEDASTICITY TESTS ---")
bp = diagnostics['heteroskedasticity']['breusch_pagan']
print(f"Breusch-Pagan: Stat={bp['statistic']:.4f}, p-value={bp['pvalue']:.4f}")

# Summary
print("\n" + "="*80)
print(f"OVERALL ASSESSMENT: {diagnostics['summary']['overall_assessment']}")
print(f"Issues detected: {diagnostics['summary']['issues_detected']}")
print("="*80)

---
## 11. Visualizations

In [ ]:
# Dynamic multipliers plot
fig = fnardl_model.plot_multipliers(horizon=24)
plt.suptitle('Dynamic Multipliers - Fourier NARDL', fontsize=14, fontweight='bold', y=1.02)
plt.savefig('dynamic_multipliers.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# CUSUM plot
fig = fnardl_model.plot_cusum()
plt.savefig('cusum_plot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# CUSUM of Squares plot
fig = fnardl_model.plot_cusumsq()
plt.savefig('cusumsq_plot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Bootstrap distributions (if available)
try:
    fig = bnardl_model.plot_bootstrap_distributions()
    plt.savefig('bootstrap_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
except:
    print("Bootstrap distribution plot not available")

In [ ]:
# Residual diagnostics
plots = NARDLPlots(fnardl_model)
fig = plots.residuals()
plt.savefig('residual_diagnostics.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 12. Results Interpretation

### Key Findings

In [ ]:
print("="*80)
print("SUMMARY OF KEY FINDINGS")
print("="*80)

print("""
1. COINTEGRATION:
   - PSS Bounds Test confirms long-run relationship exists
   - Bootstrap test provides robust inference

2. ASYMMETRIC EFFECTS:
   - Long-run asymmetry test indicates different effects of positive
     and negative changes in real economic activity on oil prices
   - Short-run dynamics also show asymmetric adjustment

3. STRUCTURAL BREAKS:
   - Fourier terms capture smooth structural changes
   - Optimal frequency k* selected by AIC

4. ERROR CORRECTION:
   - Negative ECT coefficient confirms adjustment towards equilibrium
   - Half-life indicates speed of adjustment

5. DIAGNOSTICS:
   - Model passes standard diagnostic tests
   - CUSUM tests confirm parameter stability
""")

# Specific numerical results
print("\n" + "="*80)
print("QUANTITATIVE RESULTS")
print("="*80)
print(f"\nError Correction Term (ECT): {fnardl_model.ect:.4f}")
print(f"Half-life of adjustment: {fnardl_model.half_life:.2f} periods")
print(f"R-squared: {fnardl_model.rsquared:.4f}")
print(f"Adjusted R-squared: {fnardl_model.adj_rsquared:.4f}")

---
## 13. Export Results

In [ ]:
# Export tables to LaTeX
table = ResultsTable(fnardl_model)

# Save all tables
table.to_latex('nardl_results.tex')
print("Results exported to nardl_results.tex")

# Print LaTeX long-run table
print("\n" + "="*80)
print("LATEX: LONG-RUN COEFFICIENTS")
print("="*80)
print(table.long_run_table('latex'))

In [ ]:
# Export to HTML
table.to_html('nardl_results.html')
print("Results exported to nardl_results.html")

In [ ]:
# Final watermark
print("\n" + "="*80)
print("NOTEBOOK COMPLETED")
print("="*80)
%watermark -a "Dr. Merwan Roudane" -u -d -t

---

## References

1. **Shin, Y., Yu, B., & Greenwood-Nimmo, M.** (2014). Modelling asymmetric cointegration and dynamic multipliers in a nonlinear ARDL framework. *Festschrift in Honor of Peter Schmidt*, 281-314.

2. **Kilian, L.** (2009). Not all oil price shocks are alike: Disentangling demand and supply shocks in the crude oil market. *American Economic Review*, 99(3), 1053-1069.

3. **Zaghdoudi, T., et al.** (2023). Asymmetric connectedness between oil price, coal and renewable energy consumption in China: Evidence from Fourier NARDL approach. *Energy*, 285, 129416.

4. **Bertelli, S., Vacca, G., & Zoia, M.** (2022). Bootstrap cointegration tests in ARDL models. *Economic Modelling*, 116, 105987.

---

**Author:** Dr. Merwan Roudane  
**Email:** merwanroudane920@gmail.com  
**GitHub:** https://github.com/merwanroudane/fnardl